In [59]:
with open('news', 'r') as f:
    text = f.read()
    news = text.split("\n\n")
    count = {'sport': 0, 'world': 0, "us": 0, "business": 0, "health": 0, "entertainment": 0, "sci_tech": 0}
    for news_item in news:
        lines = news_item.split("\n")
        print(lines[6])
        file_to_write = open('data/' + lines[6] + '/' + str(count[lines[6]]) + '.txt', 'w+')
        count[lines[6]] = count[lines[6]] + 1
        file_to_write.write(news_item)  # python will convert \n to os.linesep
        file_to_write.close()


sport
business
sport
entertainment
us
entertainment
world
business
business
world
us
sport
entertainment
sport
world
health
world
world
us
entertainment
sport
us
us
sport
entertainment
world
business
sport
world
sci_tech
sport
entertainment
business
us
sport
world
world
sport
us
world
sci_tech
sport
health
entertainment
business
us
entertainment
entertainment
sport
sport
entertainment
us
entertainment
business
us
sport
sport
sport
entertainment
business
business
sport
entertainment
us
world
sport
sci_tech
sport
us
sport
business
sport
us
business
us
us
us
world
entertainment
business
world
business
sport
sport
sport
business
world
health
sport
sci_tech
world
entertainment
entertainment
sport
world
world
health
health
business
world
sport
sport
sport
sport
health
world
sport
sport
world
sport
world
world
health
world
world
sport
business
us
us
world
sport
health
business
sport
sport
world
world
us
sport
us
world
business
entertainment
us
entertainment
sci_tech
us
us
business
us
entertai

IndexError: list index out of range

In [60]:
import pandas
import glob

category_list = ["sport", "world", "us", "business", "health", "entertainment", "sci_tech"]
directory_list = ["data/sport/*.txt", "data/world/*.txt","data/us/*.txt","data/business/*.txt","data/health/*.txt","data/entertainment/*.txt","data/sci_tech/*.txt",]

text_files = list(map(lambda x: glob.glob(x), directory_list))
text_files = [item for sublist in text_files for item in sublist]

training_data = []


for t in text_files:
    f = open(t, 'r')
    f = f.read()
    t = f.split('\n')
    training_data.append({'data' : t[0] + ' ' + t[1], 'flag' : category_list.index(t[6])})
    
training_data[0]

{'data': "nfl turns players' words against them in appeal to extend lockout the nfl blasted the idea that only players suffer harm during a lockout and tried to use some lockout-mitigating words of some players against ...",
 'flag': 0}

In [61]:
training_data = pandas.DataFrame(training_data, columns=['data', 'flag'])
training_data.to_csv("train_data.csv", sep=',', encoding='utf-8')
print(training_data.data.shape)


(1433,)


In [62]:
import pickle
from sklearn.feature_extraction.text import CountVectorizer


#GET VECTOR COUNT
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(training_data.data)

#SAVE WORD VECTOR
pickle.dump(count_vect.vocabulary_, open("count_vector.pkl","wb"))

In [63]:
from sklearn.feature_extraction.text import TfidfTransformer

#TRANSFORM WORD VECTOR TO TF IDF
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

#SAVE TF-IDF
pickle.dump(tfidf_transformer, open("tfidf.pkl","wb"))

In [64]:
from sklearn.metrics import confusion_matrix  

confusion_mat = confusion_matrix(y_test,predicted)
print(confusion_mat)

ValueError: Found input variables with inconsistent numbers of samples: [359, 1]

In [65]:
from sklearn import svm
from sklearn.model_selection import train_test_split
clf_svm = svm.LinearSVC()
X_train, X_test, y_train, y_test = train_test_split(X_train_tfidf, training_data.flag, test_size=0.25, random_state=42)
clf_svm.fit(X_train_tfidf, training_data.flag)
pickle.dump(clf_svm, open("svm.pkl", "wb"))

In [67]:
import pickle
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

category_list = ["sport", "world", "us", "business", "health", "entertainment", "sci_tech"]

docs_new = "Messi joins other football team"
docs_new = [docs_new]

#LOAD MODEL
loaded_vec = CountVectorizer(vocabulary=pickle.load(open("count_vector.pkl", "rb")))
loaded_tfidf = pickle.load(open("tfidf.pkl","rb"))
loaded_model = pickle.load(open("nb_model.pkl","rb"))

X_new_counts = loaded_vec.transform(docs_new)
X_new_tfidf = loaded_tfidf.transform(X_new_counts)
predicted = loaded_model.predict(X_new_tfidf)

print(category_list[predicted[0]])

sport


In [68]:
predicted = clf_svm.predict(X_test)
result_svm = pandas.DataFrame( {'true_labels': y_test,'predicted_labels': predicted})
result_svm.to_csv('res_svm.csv', sep = ',')
for predicted_item, result in zip(predicted, y_test):
    print(category_list[predicted_item], ' - ', category_list[result])

world  -  world
world  -  world
business  -  business
world  -  world
business  -  business
sport  -  sport
sport  -  sport
sport  -  sport
us  -  us
sci_tech  -  sci_tech
business  -  business
entertainment  -  entertainment
business  -  business
sport  -  sport
world  -  world
world  -  world
entertainment  -  entertainment
sport  -  sport
world  -  world
us  -  us
world  -  world
world  -  world
health  -  health
sport  -  sport
sci_tech  -  sci_tech
sport  -  sport
entertainment  -  entertainment
business  -  business
business  -  business
business  -  business
world  -  world
sport  -  sport
entertainment  -  entertainment
business  -  business
entertainment  -  entertainment
world  -  world
sport  -  sport
sport  -  sport
us  -  us
sport  -  sport
us  -  us
entertainment  -  entertainment
us  -  us
world  -  world
business  -  business
business  -  business
us  -  us
business  -  business
health  -  health
us  -  us
world  -  world
sport  -  sport
sci_tech  -  sci_tech
business  

In [69]:
from sklearn.ensemble import RandomForestClassifier
clf_random_forest = RandomForestClassifier()
X_train, X_test, y_train, y_test = train_test_split(X_train_tfidf, training_data.flag, test_size=0.25, random_state=42)
clf_random_forest.fit(X_train_tfidf, training_data.flag)
pickle.dump(clf_svm, open("random_forest.pkl", "wb"))

/home/ishaq/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [70]:
import pickle
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

category_list = ["sport", "world", "us", "business", "health", "entertainment", "sci_tech"]

docs_new = "Messi joins other football team"
docs_new = [docs_new]

#LOAD MODEL
loaded_vec = CountVectorizer(vocabulary=pickle.load(open("count_vector.pkl", "rb")))
loaded_tfidf = pickle.load(open("tfidf.pkl","rb"))
loaded_model = pickle.load(open("nb_model.pkl","rb"))

X_new_counts = loaded_vec.transform(docs_new)
X_new_tfidf = loaded_tfidf.transform(X_new_counts)
predicted = loaded_model.predict(X_new_tfidf)

print(category_list[predicted[0]])

sport


In [71]:
predicted = clf_random_forest.predict(X_test)
result_random_forest = pandas.DataFrame( {'true_labels': y_test,'predicted_labels': predicted})
result_random_forest.to_csv('random_forest.csv', sep = ',')

for predicted_item, result in zip(predicted, y_test):
    print(category_list[predicted_item], ' - ', category_list[result])

world  -  world
world  -  world
business  -  business
world  -  world
business  -  business
sport  -  sport
sport  -  sport
sport  -  sport
us  -  us
sci_tech  -  sci_tech
business  -  business
entertainment  -  entertainment
business  -  business
sport  -  sport
world  -  world
world  -  world
entertainment  -  entertainment
sport  -  sport
world  -  world
us  -  us
world  -  world
world  -  world
health  -  health
sport  -  sport
sci_tech  -  sci_tech
sport  -  sport
entertainment  -  entertainment
business  -  business
business  -  business
business  -  business
world  -  world
sport  -  sport
entertainment  -  entertainment
business  -  business
entertainment  -  entertainment
world  -  world
sport  -  sport
sport  -  sport
us  -  us
sport  -  sport
us  -  us
entertainment  -  entertainment
us  -  us
world  -  world
business  -  business
business  -  business
us  -  us
business  -  business
health  -  health
us  -  us
world  -  world
sport  -  sport
sci_tech  -  sci_tech
business  

In [72]:
from sklearn.ensemble import GradientBoostingClassifier
clf_gradient_boosting = GradientBoostingClassifier()
X_train, X_test, y_train, y_test = train_test_split(X_train_tfidf, training_data.flag, test_size=0.25, random_state=42)
clf_gradient_boosting.fit(X_train_tfidf, training_data.flag)
pickle.dump(clf_svm, open("gradient_boosting.pkl", "wb"))

In [73]:
import pickle
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

category_list = ["sport", "world", "us", "business", "health", "entertainment", "sci_tech"]

docs_new = "Messi joins other football team"
docs_new = [docs_new]

#LOAD MODEL
loaded_vec = CountVectorizer(vocabulary=pickle.load(open("count_vector.pkl", "rb")))
loaded_tfidf = pickle.load(open("tfidf.pkl","rb"))
loaded_model = pickle.load(open("nb_model.pkl","rb"))

X_new_counts = loaded_vec.transform(docs_new)
X_new_tfidf = loaded_tfidf.transform(X_new_counts)
predicted = loaded_model.predict(X_new_tfidf)

print(category_list[predicted[0]])

sport


In [74]:
predicted = clf_gradient_boosting.predict(X_test)
result_gradient_boosting = pandas.DataFrame( {'true_labels': y_test,'predicted_labels': predicted})
result_gradient_boosting.to_csv('gradient_boosting.csv', sep = ',')

for predicted_item, result in zip(predicted, y_test):
    print(category_list[predicted_item], ' - ', category_list[result])

world  -  world
world  -  world
business  -  business
world  -  world
business  -  business
sport  -  sport
sport  -  sport
sport  -  sport
us  -  us
sci_tech  -  sci_tech
business  -  business
entertainment  -  entertainment
business  -  business
world  -  sport
world  -  world
world  -  world
entertainment  -  entertainment
sport  -  sport
world  -  world
us  -  us
world  -  world
world  -  world
health  -  health
sport  -  sport
sci_tech  -  sci_tech
sport  -  sport
entertainment  -  entertainment
business  -  business
business  -  business
business  -  business
world  -  world
sport  -  sport
entertainment  -  entertainment
business  -  business
entertainment  -  entertainment
world  -  world
sport  -  sport
sport  -  sport
us  -  us
sport  -  sport
us  -  us
entertainment  -  entertainment
us  -  us
world  -  world
sport  -  business
business  -  business
us  -  us
business  -  business
health  -  health
us  -  us
world  -  world
sport  -  sport
sci_tech  -  sci_tech
business  -  